In [1]:
%pip install opencv-python-headless
%pip install scikit-learn
%pip install tensorflow==2.12.0
%pip install numpy

# Verify the installations
import cv2
import sklearn as sk
import tensorflow as tf
import numpy as np

print("cv2 version:", cv2.__version__)
print("scikit-learn version:", sk.__version__)
print("tensorflow version:", tf.__version__)
print("numpy version:", np.__version__)

# cv2 version: 4.9.0
# scikit-learn version: 1.4.2
# tensorflow version: 2.12.0
# numpy version: 1.23.5
# python 3.10.4

     -------------------------------------- 38.5/38.5 MB 482.6 kB/s eta 0:00:00
     -------------------------------------- 15.8/15.8 MB 424.4 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Owner\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


     -------------------------------------- 10.6/10.6 MB 908.8 kB/s eta 0:00:00
     -------------------------------------- 46.2/46.2 MB 466.4 kB/s eta 0:00:00
     ------------------------------------ 301.8/301.8 KB 388.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Owner\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


     ------------------------------------ 272.8/272.8 MB 556.5 kB/s eta 0:00:00
     ---------------------------------------- 5.6/5.6 MB 1.0 MB/s eta 0:00:00
     ------------------------------------ 413.4/413.4 KB 806.3 kB/s eta 0:00:00
     ------------------------------------ 440.7/440.7 KB 917.8 kB/s eta 0:00:00
     -------------------------------------- 26.4/26.4 MB 727.1 kB/s eta 0:00:00
     ------------------------------------ 133.7/133.7 KB 791.4 kB/s eta 0:00:00
     -------------------------------------- 65.5/65.5 KB 708.3 kB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 KB 1.0 MB/s eta 0:00:00
     -------------------------------------- 14.6/14.6 MB 719.8 kB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 821.4 kB/s eta 0:00:00
     ---------------------------------------- 3.0/3.0 MB 462.9 kB/s eta 0:00:00
     ---------------------------------------- 4.1/4.1 MB 506.6 kB/s eta 0:00:00
     -------------------------------------

You should consider upgrading via the 'c:\Users\Owner\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Owner\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


cv2 version: 4.9.0
scikit-learn version: 1.4.2
tensorflow version: 2.12.0
numpy version: 1.23.5


In [1]:
# data loading and proccecing (not all the proccecing )
import os
import cv2
import sklearn as sk 
import tensorflow as tf
from tensorflow.keras import layers, Sequential
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.gif')): # Adjust the extensions as per your need
            img = cv2.imread(os.path.join(folder, filename))
            if img is not None:
                images.append(img)
    return images



def one_hot_encoder(number_of_classes : int , class_num ) -> list :
    x = [0]*number_of_classes
    x[class_num] = 1
    return x 

def fitted_to_model(image) -> bool :# if image is grayscale return false
    height, width, channels = image.shape
    return channels==3 

def index_max(lst)-> int  :
    if len(lst) == 0 :
        return -1 
    i = 0 
    max = lst[0]
    for index , item in enumerate(lst) :
        if item > max :
            max = item
            i = index 
    return i 

def normelised_image(image) :
    return image/255.0

angry_x = list(filter(fitted_to_model ,load_images_from_folder("angry")))+list(filter(fitted_to_model ,load_images_from_folder("angry2")))
happy_x = list(filter(fitted_to_model ,load_images_from_folder("happy")))
relaxed_x = list(filter(fitted_to_model ,load_images_from_folder("relaxed")))
sad_x = list(filter(fitted_to_model ,load_images_from_folder("sad")))

angry_y = [0 for i in range(len(angry_x))]
happy_y = [1 for i in range(len(happy_x))]
relaxed_y = [2 for i in range(len(relaxed_x))]
sad_y = [3 for i in range(len(sad_x))]
classes_dict = {0:"angry" , 1 : "happy" , 2 : "relaxed" , 3 : "sad"}

x = angry_x + happy_x + relaxed_x + sad_x
y = angry_y + happy_y + relaxed_y + sad_y


In [2]:
import numpy as np 

# Load the model
pet_segmentation = tf.keras.models.load_model("semantic_segmentation_4_pets.keras")
# classes of the semantic segmentation model: {0:"pixcle of the dog",1:"pixcle that doesnt represent the dog (background)",2:"pixcle that represent the limit line of the dog"}
# resize images (x) from (384,384,3) to (128,128,3) to be fitted to the pet_segmentation model and normelised the values to be between 0-1
target_size = (128, 128)

x2 = [normelised_image(cv2.resize(i, target_size, interpolation=cv2.INTER_AREA)) for i in x]
print(x2[0].shape)
print(x[0].shape)


(128, 128, 3)
(384, 384, 3)


In [3]:
def filter_background(image) :
    input_image = np.reshape(image , (1, 128, 128, 3))


    prediction = pet_segmentation.predict(input_image)[0]
    h , w ,c = image.shape 
    for i in range(h) :
        for g in range(w) :
            if index_max(prediction[i,g]) == 1 :
                image[i,g] = [0,0,0]

# delete all the background from the normelised images (making them black)
for i in range(len(x2)) :
    filter_background(x2[i])
    print(i+1)
 

1/1 [==============================] - 1s 1s/step
1
1/1 [==============================] - 0s 47ms/step
2
1/1 [==============================] - 0s 47ms/step
3
1/1 [==============================] - 0s 31ms/step
4
1/1 [==============================] - 0s 31ms/step
5
1/1 [==============================] - 0s 31ms/step
6
1/1 [==============================] - 0s 31ms/step
7
1/1 [==============================] - 0s 31ms/step
8
1/1 [==============================] - 0s 47ms/step
9
1/1 [==============================] - 0s 47ms/step
10
1/1 [==============================] - 0s 47ms/step
11
1/1 [==============================] - 0s 47ms/step
12
1/1 [==============================] - 0s 47ms/step
13
1/1 [==============================] - 0s 53ms/step
14
1/1 [==============================] - 0s 47ms/step
15
1/1 [==============================] - 0s 41ms/step
16
1/1 [==============================] - 0s 47ms/step
17
1/1 [==============================] - 0s 31ms/step
18
1/1 [================

In [4]:
from sklearn.model_selection import train_test_split
# del x # makes room for more variabels in order to not have memoryerror
 
y = [one_hot_encoder(4,i) for i in y]
x3 = np.array(x2)
y = np.array(y)

x_train , x_test ,y_train , y_test = train_test_split(x3,y,test_size = 0.2 , random_state = 42 )


In [7]:
from keras.models import Sequential
from keras import layers, regularizers
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16

# Define the image data generator with augmentation, without rescaling since data is already normalized
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,  # Changed to True for more augmentation
    fill_mode='nearest'
)

# Validation data generator without augmentation
val_datagen = ImageDataGenerator()

# Assuming x_train, y_train, x_val, y_val are already defined
train_generator = train_datagen.flow(x_train, y_train, batch_size=64)
val_generator = val_datagen.flow(x_test, y_test, batch_size=64)

# Load the VGG16 model with pretrained weights, excluding the top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
num_classes = 4 #outputlayer shape, number of classes
# Freeze the first few layers
for layer in base_model.layers[:15]:  # Adjust the number of layers to freeze
    layer.trainable = False

# Add custom top layers
model = Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))

# Define learning rate schedule function
def lr_schedule(epoch, lr):
    if epoch not in  [11,16,25,35]:
        return lr  # No change for the first 10 epochs
    else:
        return lr * 0.5  # Decrease learning rate by a factor of 0.5 after the first 10 epochs

# Create Adam optimizer with initial learning rate
adam_optimizer = Adam(learning_rate=0.0001)  # Lower initial learning rate

# Define learning rate scheduler callback
lr_scheduler = LearningRateScheduler(lr_schedule)

# Compile the model
model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
model.fit(train_generator, validation_data=val_generator, epochs=50, callbacks=[early_stopping, lr_scheduler])

# Save the model
model.save("pets_emotion_clc_vgg16_finetuned2.keras")


Epoch 1/50
212/212 [==============================] - 3061s 14s/step - loss: 1.4388 - accuracy: 0.3234 - val_loss: 1.3287 - val_accuracy: 0.4417 - lr: 1.0000e-04
Epoch 2/50
212/212 [==============================] - 3059s 14s/step - loss: 1.3554 - accuracy: 0.3881 - val_loss: 1.2774 - val_accuracy: 0.4570 - lr: 1.0000e-04
Epoch 3/50
212/212 [==============================] - 3064s 14s/step - loss: 1.3256 - accuracy: 0.4141 - val_loss: 1.2553 - val_accuracy: 0.4677 - lr: 1.0000e-04
Epoch 4/50
212/212 [==============================] - 3083s 15s/step - loss: 1.3005 - accuracy: 0.4345 - val_loss: 1.2452 - val_accuracy: 0.4783 - lr: 1.0000e-04
Epoch 5/50
212/212 [==============================] - 3090s 15s/step - loss: 1.2813 - accuracy: 0.4447 - val_loss: 1.2346 - val_accuracy: 0.4824 - lr: 1.0000e-04
Epoch 6/50
212/212 [==============================] - 3098s 15s/step - loss: 1.2595 - accuracy: 0.4559 - val_loss: 1.2189 - val_accuracy: 0.5004 - lr: 1.0000e-04
Epoch 7/50
212/212 [========

In [8]:
# turnning the segmentation tensorfllow model into a tensorfllow lite model for using in android project 
import tensorflow as tf
def convert_model_2_lite(model_path : str ,lite_model_name : str ) -> None :
    # Load your TensorFlow model
    model = tf.keras.models.load_model(model_path)

    # Optimize the model if necessary
    # For example, you can quantize the model
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model = converter.convert()

    # Save the TensorFlow Lite model to a file
    with open(lite_model_name+'.tflite', 'wb') as f:
        f.write(tflite_model)

convert_model_2_lite("pets_emotion_clc_vgg16_finetuned2.keras","pets_emotion_classification_finetune2")

INFO:tensorflow:Assets written to: C:\Users\Owner\AppData\Local\Temp\tmpe6qyy4rx\assets


INFO:tensorflow:Assets written to: C:\Users\Owner\AppData\Local\Temp\tmpe6qyy4rx\assets
